In [ ]:
import pandas as pd 
import numpy as np 
import datetime as dt
import seaborn as sns 
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import STL
from pmdarima import auto_arima
from statsmodels.tsa.arima.model import ARIMA
from arch import arch_model
import pickle 

In [ ]:
# Read in Data
corporations = pd.read_excel("data/All_Corps_and_VotingUnits.xlsx", index_col="Date").iloc[:,3:10]
corporations.columns = ["name", "email", "status", "CAO", "CMRAO", "account_id2", "voting_units"]
corporations = corporations.sort_index()

In [ ]:
# Daily Units 
plt.figure(figsize = (20,6))
plt.plot(corporations['voting_units'])
plt.title("Distriution of Daily Unit Counts")
plt.ylabel("Total Voting Units")

In [ ]:
# Not a lot of meaningful data before 1965
corporations = corporations[(corporations.index > dt.datetime(1967,1,1)) & (corporations.index <= dt.datetime(2025,11,1))]

In [ ]:
# Daily Units - including filter
plt.figure(figsize = (20,6))
plt.plot(corporations['voting_units'])
plt.title("Distriution of Daily Unit Counts")
plt.ylabel("Total Voting Units")

In [ ]:
# resampling data, going with monthly aggregations for predicitions 

resampled_df = corporations.resample('ME').sum(numeric_only=True)
#resampled_df = resampled_df.fillna(0)
window_size = 40
resampled_df['units_smoothed'] = resampled_df['voting_units'].rolling(window=window_size).mean() # smoothed/moving average
resampled_df['units_smoothed_sd'] = resampled_df['voting_units'].rolling(window=window_size).std() # smoothed/moving sd

In [ ]:
# Plotting things to look at the nature of the data 
plt.figure(figsize=(20,6))
sns.lineplot(data=resampled_df, x = resampled_df.index, y='voting_units', label = "Resampled Monthly Sum")
sns.lineplot(data=resampled_df, x = resampled_df.index, y='units_smoothed', label = "Moving Average")
sns.lineplot(data=resampled_df, x = resampled_df.index, y='units_smoothed_sd', label = "Moving Std. Deviation")
plt.xlabel("Registration Date")
plt.ylabel("Total Voting Units")
plt.title("Total Units, Moving Average/Std. Dev Over Time")
plt.show()

In [ ]:
# Variance is not constant, neither is the mean
# dataset is likely not stationary. Will confirm this using dickey fuller test 
result = adfuller(resampled_df['voting_units'].dropna())
print('ADF:', result[0])
print('p:', result[1])
print('Critical Values:', result[4])
if result[1] <= 0.05: print("The dataset is stationary") 
else: print("The dataset is not stationary")

In [ ]:
# Identifying Seasonality Trends
plt.figure(figsize=(20,6))
plt.title("STL Trend")
res = STL(resampled_df['voting_units'].dropna(), period=12).fit()
res.resid.plot()

# there are no seasonality trends (clear, repeating patterns in the data)

In [ ]:
resampled_df['vu_log'] = np.log1p(resampled_df['voting_units'].dropna()) # stabilizing the variance of the dataset 
resampled_df['vu_diff'] = resampled_df.vu_log.diff().dropna()
resampled_df

In [ ]:
# dataset should be stationary after log transform and differencing 
result = adfuller(resampled_df['vu_diff'].dropna())
print('ADF:', result[0])
print('p:', result[1])
print('Critical Values:', result[4])
if result[1] <= 0.05: print("The dataset is stationary") 
else: print("The dataset is not stationary")

In [ ]:
time_series = resampled_df.vu_diff.dropna()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# ACF plot (autocorrelation)
plot_acf(time_series, ax=axes[0], lags=40)
axes[0].set_title('ACF Plot')

# PACF plot (partial autocorrelation)
plot_pacf(time_series, ax=axes[1], lags=40, method='ywm')
axes[1].set_title('PACF Plot')

plt.show()

In [ ]:
# ACF has big negative spike at lag 1, which is a signature of an MA(1) proces. 
# PACF is telling us strong neg spike at lag 1, matches AR(1) or AR(2) pattern. Not as strongly 
# combined, AIRMA 0, 1,1 seems like the most natural model 
# ACF and PACF have a dominant lag-1 effect, but the ACF has clean sharp spike 
# differencing was needed, noise is MA(1) 
# no seasonality needs to beincluded 

# ARIMA(0,1,1)

In [ ]:
model = ARIMA(time_series, order=(0,1,1))
result = model.fit()
print(result.summary())
# terrible, terrible model as ther eis a lot of volatility in the dataset 

# GARCH(1,1)

In [ ]:
garch = arch_model(time_series, vol='GARCH', p=1, q=1, mean='Zero')
garch_results = garch.fit()
print(garch_results.summary())
# not as great, but we are getting there 

# ARIMA(0,1,1) + GARCH(1,1)

In [ ]:
# Fit ARIMA(0,1,1) first
arima_model = ARIMA(resampled_df["voting_units"].dropna(), order=(0,1,1))
arima_res = arima_model.fit()

# Get ARIMA residuals to feed into GARCH
arima_resid = arima_res.resid.dropna()

# Fit GARCH on ARIMA residuals
garch = arch_model(arima_resid, vol='GARCH', p=1, q=1, mean='Zero')
garch_res = garch.fit()

print(garch_res.summary())

In [ ]:
garch_res.plot()


In [ ]:
auto_arima(resampled_df['voting_units'].dropna(), seasonal=False, stepwise=True) # also an option for less volatile data 

In [ ]:
# Save models

models = {
    "arima": arima_model,
    "garch": garch_res
}

with open("arima_garch_bundle.pkl", "wb") as f:
    pickle.dump(models, f)